### Tests GPU

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Load data

In [2]:
from utils.process_tensors import load_tensor, normalize_tensor
import tensorflow as tf

# Parameters
loss = 'mae'
nb_layers = 3
nb_filters = 32
batch_size = 64
shape = (8,8,8)

train_ratio = .8

random_boards_files = ['random1', 'random2', 'random3', 'random4']
from_pgn_files = ['lichess1', 'lichess2', 'lichess3', 'lichess4', 'lichess5','lichess6', 'lichess7']
file_names = from_pgn_files + random_boards_files

input_tensor = None
output_tensor = None
for file_name in file_names:
    load_path = f"./data/{file_name}/"
    input_path = load_path + "input"
    output_path = load_path + "output"
    x = load_tensor(input_path, out_type=tf.int32)
    y = load_tensor(output_path, out_type=tf.float32)
    if input_tensor is None:
        input_tensor = x
        output_tensor = y
    else:
        input_tensor = tf.concat([input_tensor, x], axis=0)
        output_tensor = tf.concat([output_tensor, y], axis=0)

print('input tensor shape:', input_tensor.shape)
print('output tensor shape:', output_tensor.shape)


input tensor shape: (205000, 8, 8, 8)
output tensor shape: (205000,)


### Transform Data

In [3]:
from utils.process_tensors import transform_square_root_tensor, reverse_transform_square_root_tensor

max_value = tf.reduce_max(tf.abs(output_tensor)).numpy()
transformed_tensor = transform_square_root_tensor(output_tensor, max_value=1.0)


### Visualize data

In [4]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(20, 2.5))
bins = 100
axs[0].hist(output_tensor, bins=bins)
axs[1].hist(transformed_tensor, bins=bins)
print(f'max value of output_tensor: {max_value}')


### Make Dataset + separate into train and test

In [60]:
dataset:tf.data.Dataset = tf.data.Dataset.from_tensor_slices((input_tensor, transformed_tensor))
print('Number of positions:',dataset.cardinality().numpy())

# batch and shuffle
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(1)
dataset = dataset.shuffle(buffer_size=dataset.cardinality().numpy())

# split
train_size = int(dataset.cardinality().numpy() * train_ratio)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

print('Number of batches:',batch_size)
print('train size:',train_dataset.cardinality().numpy())
print('test size:',test_dataset.cardinality().numpy())

Number of positions: 205000
Number of batches: 64
train size: 2563
test size: 641


### Model

In [61]:
from utils.build_model import build_model

model = build_model(nb_filters=nb_filters , nb_layers=nb_layers, shape=shape)
model.compile(optimizer='adam', loss=loss, metrics=['mae', 'mse', 'msle'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 8, 8, 32)          2336      
                                                                 
 dropout_4 (Dropout)         (None, 8, 8, 32)          0         
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 32)          9248      
                                                                 
 dropout_5 (Dropout)         (None, 8, 8, 32)          0         
                                                                 
 conv2d_6 (Conv2D)           (None, 8, 8, 32)          9248      
                                                                 
 dropout_6 (Dropout)         (None, 8, 8, 32)          0         
                                                                 
 flatten_2 (Flatten)         (None, 2048)             

In [63]:
import os
from utils.callback_setup import callback_setup, create_checkpoint_name
# checkpoint_name = f"{loss}_{nb_layers}_{nb_filters}_{batch_size}_02/"

checkpoint_name = create_checkpoint_name(loss=loss, nb_layers=nb_layers, nb_filters=nb_filters, batchsize=batch_size, shape=shape, try_nb=1)
print('checkpoint name:',checkpoint_name)
logs_dir = f'logs/{checkpoint_name}/'
checkpoint_dir = os.path.join('backup', checkpoint_name)

model_callbacks = callback_setup(checkpoint_dir, logs_dir, monitor='loss', mode='min')

checkpoint name: loss=mse-layers=3-filters=32-batchsize=64-shape=888-try=01


### Load Weights

In [67]:
# checkpoint_name = "loss=mse-layers=3-filters=32-batchsize=16-shape=888-try=01"
try:
    checkpoint_filepath = os.path.join('backup', checkpoint_name)

    model.load_weights(checkpoint_filepath)
except:
    print("No checkpoint found.")
    pass

No checkpoint found.


In [54]:
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=(test_dataset),
                    verbose=1,
                    callbacks=model_callbacks
                    )

Epoch 1/50
1281/1281 [==============================] - 13s 8ms/step - loss: 0.0586 - mae: 0.1606 - mse: 0.0586 - msle: 0.0138 - val_loss: 0.0499 - val_mae: 0.1472 - val_mse: 0.0499 - val_msle: 0.0135
Epoch 2/50
1281/1281 [==============================] - 13s 10ms/step - loss: 0.0487 - mae: 0.1445 - mse: 0.0487 - msle: 0.0116 - val_loss: 0.0454 - val_mae: 0.1379 - val_mse: 0.0454 - val_msle: 0.0099
Epoch 3/50
1281/1281 [==============================] - 13s 10ms/step - loss: 0.0467 - mae: 0.1403 - mse: 0.0467 - msle: 0.0111 - val_loss: 0.0430 - val_mae: 0.1352 - val_mse: 0.0430 - val_msle: 0.0105
Epoch 4/50
1281/1281 [==============================] - 12s 9ms/step - loss: 0.0456 - mae: 0.1385 - mse: 0.0456 - msle: 0.0109 - val_loss: 0.0422 - val_mae: 0.1310 - val_mse: 0.0422 - val_msle: 0.0099
Epoch 5/50
1281/1281 [==============================] - 13s 10ms/step - loss: 0.0447 - mae: 0.1365 - mse: 0.0447 - msle: 0.0107 - val_loss: 0.0414 - val_mae: 0.1297 - val_mse: 0.0414 - val_msle:

KeyboardInterrupt: 

### Test on data from the training set

In [46]:
from utils.process_tensors import reverse_transform_square_root_tensor
import random

k = random.randrange(0, 1000)
n = 1

x_test = input_tensor[k:k+n,:,:,:]
y_test = transformed_tensor[k:k+n]

y_pred = model.predict(x_test, verbose=0)

print("---- With transformation ----")
print("true score: ", y_test.numpy())
print("predicted score: ", y_pred[:,0])

print("\n---- Real score ----")
print("true score: ", reverse_transform_square_root_tensor(y_test, max_value).numpy())
print("predicted score: ", reverse_transform_square_root_tensor(y_pred[:,0], max_value).numpy())

---- With transformation ----
true score:  [0.08426572]
predicted score:  [0.1292356]

---- Real score ----
true score:  [0.71000004]
predicted score:  [1.6700169]


### Tests on random board

In [38]:
from utils.random_board import generate_random_board
from utils.process_board import convert_board_to_array
from utils.stockfish import get_score

board = generate_random_board()
score = get_score(board)
x_test = tf.convert_to_tensor(convert_board_to_array(board, shape=shape))
x_test = tf.expand_dims(x_test, axis=0)

y_pred = model.predict(x_test)

print("predicted transformed score: ", y_pred[0][0])

print("\nTrue score: ", score)
print("Predicted score: ", reverse_transform_square_root_tensor(y_pred, max_value).numpy()[0][0])


1/1 [==============================] - 0s 31ms/step
predicted transformed score:  0.8371248

True score:  39.01
Predicted score:  70.070786
